In [102]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [103]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH12-RAG")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH12-RAG


In [104]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader,PDFPlumberLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [105]:
# 단계 1: 문서 로드(Load Documents)
loader = PyMuPDFLoader("./Afterschool_Adventure_Time-20-29.pdf")
docs = loader.load()

In [106]:
docs[0].metadata

{'source': './Afterschool_Adventure_Time-20-29.pdf',
 'file_path': './Afterschool_Adventure_Time-20-29.pdf',
 'page': 0,
 'total_pages': 10,
 'format': 'PDF 1.4',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'creator': '',
 'producer': 'iLovePDF',
 'creationDate': '',
 'modDate': 'D:20240714044133Z',
 'trapped': ''}

In [107]:
# 단계 2: 문서 분할(Split Documents)
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30)
split_documents = text_splitter.split_documents(docs)

In [108]:
# 단계 3: 임베딩(Embedding) 생성
embeddings = OpenAIEmbeddings()

In [109]:
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다. (메모리에 저장하는것이고 아직 디스크에 저장하는 것은 아님)
vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

In [110]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
retriever = vectorstore.as_retriever()

In [111]:
# retriever.invoke("시나리오 개요 작성해줘")

In [112]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """당신은 30년 차 TRPG 게임 사회자입니다.
현재 "방과후 탐사활동기록" 시나리오의 사회자를 맡게 되었습니다.
Player들의 선택지를 참고하여 시나리오를 진행하려고 합니다.
다음과 같은 검색된 context를 사용하여 시나리오를 자연스럽게 만들어주세요.
답변은 한글로 작성하세요.

다음 사항을 준수하세요:
- 답변은 상황에 대한 시나리오 진행과 3가지 선택지 제공으로 구성됩니다.
- 답변 형식은 [예제]와 똑같이 작성해주세요.
- 시나리오 진행 도중 주사위를 굴려야 하는 상황이 온다면 플레이어에게 주사위를 굴리라고 안내해주고 주사위 입력값을 기다립니다.
- 주사위 입력값이 들어오면 플레이어의 특성값을 확인하여 결과를 알려주고 시나리오를 진행합니다.

# 예제:
    여러분은 조심스럽게 2층으로 올라가기로 합니다. 계단을 오르면서 소리가 점점 더 크게 들려옵니다. 2층에 도착하자, 소리는 복도 끝에 있는 문 뒤에서 나는 것 같습니다. 문은 살짝 열려있고, 안쪽에서 희미한 빛이 새어나오고 있습니다.
    이제 여러분은 다음과 같은 선택지를 선택할 수 있습니다.

    **선택지:**
    1. 문을 열어본다.
    2. 문을 열지 않고 돌아간다.
    3. 다른 방향으로 이동한다.

#Question: 
{question} 

#Context: 
{context} 

#Answer:"""
)

In [113]:
# 단계 7: 언어모델(LLM) 생성
# 모델(LLM) 을 생성합니다.
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [114]:
strength = [10, 20, 30, 40]
health = [100, 200, 300, 400]
size = [1, 2, 3, 4]
agility = [5, 10, 15, 20]
look = [6, 12, 18, 24]
education = [2, 4, 6, 8]
iq = [110, 120, 130, 140]
mental = [1, 2, 3, 4]

In [115]:
# 단계 8: 체인(Chain) 생성
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [118]:
# 체인 실행(Run Chain)
# 문서에 대한 질의를 입력하고, 답변을 출력합니다.
content = "2. 그들을 무시하고 학교 안으로 들어간다."
response = chain.invoke(content)
print(response)

여러분은 교문 앞의 사람들을 무시하고 학교 안으로 들어가기로 합니다. 학교 안으로 들어가자, 운동장 구석의 나무 뒤 풀 숲이 기묘하게 흔들리는 소리가 들립니다. 그쪽으로 다가가면, 검은색 무언가가 빠르게 튀어나와 샤샤샥 학교 뒤로 뛰어가는 모습을 보게 됩니다. 

이제 여러분은 다음과 같은 선택지를 선택할 수 있습니다.

**선택지:**
1. 검은색 무언가를 따라간다.
2. 학교 안으로 더 깊이 들어간다.
3. 운동장을 둘러본다.
